In [25]:
# libraries
import datetime
import time
import json
from pymongo import MongoClient
from faker import Faker

# Data generation

In [26]:
class Lab2models:
    def data_generator(self, num_companies, people_per_company):
        # Connect to MongoDB - Note: added my local host
        client = MongoClient('mongodb://localhost:27017/')
        db = client['lab2']
        # delete collection data if exists from all fo the 3 models
        db.drop_collection("m1_people")
        db.drop_collection("m1_companies")
        db.drop_collection("m2_people")
        db.drop_collection("m3_companies")
        # create and obtain collections
        m1_people = db.create_collection('m1_people')
        m1_companies = db.create_collection('m1_companies')
        m2_people = db.create_collection('m2_people')
        m3_companies = db.create_collection('m3_companies')
        # It will generate data in spanish: 'es_ES'
        fake = Faker(['es_ES'])
        # Generate company data
        company_data = []
        for i in range(num_companies):
            company_name = fake.company()
            domain = fake.domain_name()
            company = {
                "domain": domain,
                "email": f"info@{domain}",
                "name": company_name,
                "url": f"www.{domain}",
                "vatNumber": fake.bothify(text='??######')
            }
            company_data.append(company)
        
        # Insert companies for Model 1
        company_ids = m1_companies.insert_many(company_data).inserted_ids

        # Generate people's data
        person_data = []
        for i in range(num_companies * people_per_company):
            company_idx = i % num_companies  # Distribute the people among companies
            
            if fake.random_element(elements=(True, False)):
                first_name = fake.first_name_male()
                sex = 'M'
            else:
                first_name = fake.first_name_female()
                sex = 'F'
                
            last_name = fake.last_name()
            full_name = f"{first_name} {last_name}"
            birth_date = fake.date_of_birth(minimum_age=20, maximum_age=65)
            birth_datetime = datetime.datetime.combine(birth_date, datetime.time())
            age = (datetime.datetime.now().date() - birth_date).days // 365
            
            username_full = f"{first_name.lower()}.{last_name.lower()}".replace(" ", "")
            username_short = f"{first_name.lower()[0]}{last_name.lower()}".replace(" ", "")
            
            person = {
                "age": age,
                "companyEmail": f"{username_full}@{company_data[company_idx]['domain']}",
                "dateOfBirth": birth_datetime,
                "email": f"{username_short}@{fake.free_email_domain()}",
                "firstName": first_name,
                "fullName": full_name,
                "sex": sex  # Ahora el sexo siempre coincide con el nombre
            }
            person_data.append((person, company_idx))

        # MODEL 1
        for i, (person, company_idx) in enumerate(person_data):
            m1_person = person.copy()
            m1_person["worksIn"] = company_ids[company_idx]  # Reference to company
            m1_people.insert_one(m1_person)
        
        # MODEL 2
        for i, (person, company_idx) in enumerate(person_data):
            m2_person = person.copy()
            m2_person["worksIn"] = company_data[company_idx]  # Embed company document
            m2_people.insert_one(m2_person)
        
        # MODEL 3
        persons_by_company = {}
        for person, company_idx in person_data:
            if company_idx not in persons_by_company:
                persons_by_company[company_idx] = []
            persons_by_company[company_idx].append(person.copy())
        
        for i, company in enumerate(company_data):
            m3_company = company.copy()
            m3_company["staff"] = persons_by_company.get(i, [])
            m3_companies.insert_one(m3_company)
        
        return num_companies, people_per_company
        
    def close_connection(self):
        if hasattr(self, 'client'):
            self.client.close()

In [27]:
if __name__ == "__main__":
    lab = Lab2models()

    num_companies = 30       
    people_per_company = 100  
    
    print(f"Generating data for {num_companies} companies with approximately {people_per_company} people per company...")
    print(f"Total expected documents: {num_companies} companies and {num_companies * people_per_company} people")
    
    start_time = time.time()
    lab.data_generator(num_companies, people_per_company)
    
    generation_time = time.time() - start_time
    
    print(f"\nData generation completed in {generation_time:.2f} seconds")

Generating data for 30 companies with approximately 100 people per company...
Total expected documents: 30 companies and 3000 people

Data generation completed in 0.84 seconds


# Queries